In [ ]:
import numpy as np
import h5py
import itertools
import matplotlib.pyplot as plt
from corner import corner
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_probability as tfp

tfk = tf.keras
tfd = tfp.distributions
tfb = tfp.bijectors

seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

In [ ]:
def sample_powerlaw(n_samples, slope, lo, hi):
    
    x = np.random.uniform(size=n_samples)
    
    return (lo**(slope+1) + x * (hi**(slope+1) - lo**(slope+1)))**(1/(slope+1))

def sample_data(n_dim, n_samples, slope, lo, hi, loc, scale, bounded_idxs=[1, 2]):
    
    n_bounded = len(bounded_idxs)
    n_unbounded = n_dim - n_bounded
    unbounded_idxs = list(set(range(n_dim)) - set(bounded_idxs))
    data = np.zeros([n_samples, n_dim])
    data[:, unbounded_idxs] = np.random.normal(
        loc=loc, scale=scale, size=[n_samples, n_unbounded],
        )
    data[:, bounded_idxs] = sample_powerlaw(
        [n_samples, n_bounded], slope, lo, hi,
        )
    
    return data

In [ ]:
n_dim = 2
n_samples = 10000
slope = 1
lo = 0
hi = 1
loc = 2
scale = 3
bounded_idxs = [1]

data = sample_data(n_dim, n_samples, slope, lo, hi, loc, scale, bounded_idxs)

corner(data);

In [ ]:
# Need to permute between flows
# For n_dim > 2, there are multiple choices
# But we have to ensure all outputs are moved

def all_permuted(n_dim):
    
    permutations = np.array(list(itertools.permutations(range(n_dim))))
    permuted = ~np.any(permutations == list(range(n_dim)), axis=1)
    
    return permutations[permuted]

In [ ]:
permutations = all_permuted(n_dim)
permutations

In [ ]:
n_flows = 5
n_layers = 1
n_neurons = 128
activation = 'relu' #tfa.activations.rrelu

bijectors = []

# We transform the bounded outputs at the end
# Adjust shift and scale to whatever domain you need
blockwise_bijectors = []
for i in range(n_dim):
    if i in bounded_idxs:
        sigmoid = tfb.Chain(
            #[tfb.Tanh(), tfb.Shift(shift=1.), tfb.Scale(scale=.5)],
            [tfb.Scale(scale=.5), tfb.Shift(shift=1.), tfb.Tanh()],
            )
        blockwise_bijectors.append(sigmoid)
    else:
        blockwise_bijectors.append(tfb.Identity())
blockwise_bijector = tfb.Blockwise(blockwise_bijectors, block_sizes=[1]*n_dim)
bijectors.append(blockwise_bijector)

for i in range(n_flows):
    bijectors.append(tfb.MaskedAutoregressiveFlow(tfb.AutoregressiveNetwork(
        params=2,
        hidden_units=[n_neurons]*n_layers,
        activation=activation,
        )))
    #bijectors.append(tfb.BatchNormalization(training=True))
    bijectors.append(tfb.Permute(list(reversed(range(n_dim)))))

bijector = tfb.Chain(bijectors)
distribution = tfd.MultivariateNormalDiag(loc=[0]*n_dim)
nf = tfd.TransformedDistribution(distribution=distribution, bijector=bijector)

In [ ]:
# Check the correct dimensions are bounded
nf.sample(10000).numpy().min(axis=0), nf.sample(10000).numpy().max(axis=0)

In [ ]:
epochs = 10
batch_size = 100
learning_rate = 1e-3

x = tf.keras.Input(shape=[n_dim], dtype=tf.float32)
log_prob = nf.log_prob(x)

model = tf.keras.Model(
    inputs=x,
    outputs=log_prob,
    )

model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=learning_rate),
    loss=lambda _, log_prob: -log_prob,
    )

# callbacks = [
#     tfk.callbacks.ModelCheckpoint(
#         filepath='./5_nonconditional.hdf5',
#         monitor='val_loss',
#         mode='min',
#         save_weights_only=True,
#         save_best_only=True,
#         save_freq='epoch',
#         verbose=1,
#         ),
#     tfk.callbacks.CSVLogger(
#         './nf2.csv',
#         ),
#     tfk.callbacks.EarlyStopping(
#         monitor='val_loss',
#         min_delta=0,
#         patience=10,
#         mode='min',
#         baseline=None,
#         restore_best_weights=False,
#         verbose=1,
#         ),
#     ]

steps_per_epoch = n_samples // batch_size

result = model.fit(
    x=data, y=np.zeros(n_samples),
    validation_data=[
        sample_data(n_dim, n_samples, slope, lo, hi, loc, scale, bounded_idxs),
        np.zeros(n_samples),
        ],
    epochs=epochs,
    batch_size=batch_size,
    steps_per_epoch=steps_per_epoch,
    shuffle=True,
    verbose=1,
    #callbacks=callbacks,
    )

In [ ]:
plt.plot(result.history['loss'])
plt.plot(result.history['val_loss']);

In [ ]:
# Should be like the training data
corner(nf.sample(n_samples).numpy());

In [ ]:
# Should be a Gaussian
samples = sample_data(
    n_dim, n_samples, slope, lo, hi, loc, scale, bounded_idxs,
    )
corner(bijector.inverse(samples).numpy());